In [ ]:
%pip install -r requirements.txt

In [ ]:
from typing import Any, Dict, List
import os
from dotenv import load_dotenv
load_dotenv()


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
)

### Überblick
In diesem Projekt simulieren mehrere KI-Agenäten einen lebendigen Buchclub. Sie diskutieren, welches Buch als nächstes gemeinsam gelesen werden soll. Jeder Agent bringt eigene Perspektiven ein, reagiert auf die Argumente der anderen und beteiligt sich aktiv an der Entscheidungsfindung. Ziel ist es, einen diskursiven, dynamischen Gesprächsverlauf zu ermöglichen – ganz wie unter echten Menschen

In [41]:
from random import randrange

def suche_bewertung(buchtitel: str):
    sterne = randrange(3, 6) #Zw. 3 und 5
    return f"{buchtitel}: {sterne} von 5 Sterne"

In [42]:
system_message_template = """
    Die Abbruchbedingung der Diskussion ist 'FERTIG'.
    Schreibt das also nicht nach jeder Nachricht sondern nur,
    wenn sich alle auf ein Buch einig sind.
    Sollte das Wort davor schon in genau dieser Schreibweise genannt werden,
    wird das Gespräch abgebrochen. 
"""

leser = AssistantAgent(
    "leser",
    model_client=model_client,
    handoffs=["kritiker", "suchender"],
    system_message= system_message_template + """
        **Rolle:** Inspirator und Empfehler.
        **Verhalten:** Bringt einen oder mehrere Buchtitel ins Spiel. Begründet seine Vorschläge mit persönlichen Eindrücken oder Highlights („Ich habe neulich XYZ gelesen, das war spannend, weil ...“).
        **Initiative:** Stellt auch gerne Rückfragen (z.B. „Kennt ihr das auch?“, „Würde euch das interessieren?“), reagiert aber auch auf Gegenargumente, indem er näher erklärt, warum das Buch lesenswert sei.
    """
)

kritiker = AssistantAgent(
    "kritiker",
    model_client=model_client,
    handoffs=["leser", "suchender"],
    tools=[suche_bewertung],
    system_message= system_message_template + """
        **Rolle:** Analyst und Verfechter von Qualität.
        **Verhalten:** Prüft die Vorschläge kritisch, bringt ggf. eigene Erfahrungen ein, stellt starke oder schwache Argumente fest.
        **Dynamik:** Erwartung ist, dass er Rückfragen stellt („Was ist das Besondere an diesem Buch?“), Alternativen einwirft oder Schwächen aufdeckt („Das Thema ist mir zu nischig“, „Ich fand den Schreibstil anstrengend“). Wenn überzeugt, kann er dem Vorschlag auch zustimmen.
    """
)

suchender = AssistantAgent(
    "suchender",
    model_client=model_client,
    handoffs=["leser", "kritiker"],
    system_message= system_message_template + """
       **Rolle:** Der Neugierige und Vergleichende.
       **Verhalten:** Sucht aktiv nach weiteren Optionen, bringt ggf. ähnliche oder kontrastierende Buchtitel ein.
       **Interaktion:** Fordert Diskussionspartner zu Vergleichen heraus („Findet ihr Buch A oder B spannender?“). Kann vermittelnd sein oder gezielt die Diskussion offen halten („Wir könnten auch aus dem Genre X etwas nehmen – was meint ihr dazu?“).
    """
)


### Abbruchbedingung 

In [43]:
termination = HandoffTermination(target="user") | TextMentionTermination("FERTIG")

### Erstellung des Teams & Start der Diskussion

In [44]:
team = Swarm([leser, kritiker, suchender], termination_condition=termination)

In [ ]:
task = """
    Sucht ein Buch, das in diesem Buchclub als nächstes gelesen werden soll.
    Arbeitet darauf hin, dass ihr euch.
"""

async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")

        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]

await run_team_stream()
await model_client.close()